In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
train_generator = train_datagen.flow_from_directory(
        '../data/archive/train/mouth',  # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=16)

validation_generator = train_datagen.flow_from_directory(
        '../data/archive/val/mouth',  # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=16)

Found 1069 images belonging to 2 classes.
Found 379 images belonging to 2 classes.


In [17]:
base_model = tf.keras.applications.InceptionResNetV2(
    include_top=False, weights='imagenet', input_shape=(224,224,3)
)

In [24]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    include_top=False, weights='imagenet', input_shape=(224,224,3)
)

In [23]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [24]:
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [25]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(
      train_generator,
      validation_data = validation_generator,
      epochs=20,
      verbose=1,
      callbacks=early_stop
      )

Epoch 1/20
67/67 [==============================] - 81s 716ms/step - loss: 0.1191 - accuracy: 0.9519 - val_loss: 3.0864 - val_accuracy: 0.9393
Epoch 2/20
67/67 [==============================] - 42s 620ms/step - loss: 0.0621 - accuracy: 0.9858 - val_loss: 0.7472 - val_accuracy: 0.9208
Epoch 3/20
67/67 [==============================] - 42s 628ms/step - loss: 0.0380 - accuracy: 0.9901 - val_loss: 0.5742 - val_accuracy: 0.9314
Epoch 4/20
67/67 [==============================] - 45s 672ms/step - loss: 0.0170 - accuracy: 0.9955 - val_loss: 6.2510 - val_accuracy: 0.5989
Epoch 5/20
67/67 [==============================] - 74s 1s/step - loss: 0.0416 - accuracy: 0.9831 - val_loss: 4.8344 - val_accuracy: 0.8602
Epoch 6/20
67/67 [==============================] - 99s 1s/step - loss: 0.0413 - accuracy: 0.9886 - val_loss: 0.1472 - val_accuracy: 0.9842
Epoch 7/20
67/67 [==============================] - 61s 914ms/step - loss: 0.0045 - accuracy: 0.9988 - val_loss: 0.0654 - val_accuracy: 0.9868
Epoch

In [42]:
test_generator = train_datagen.flow_from_directory(
        '../data/test/mouth',  # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=16)

Found 62 images belonging to 2 classes.


In [43]:
model.evaluate(test_generator)

4/4 [==============================] - 5s 1s/step - loss: 1.4084 - accuracy: 0.7581


[1.4084243774414062, 0.7580645084381104]